In [ ]:
m

# **Carregar conjunto de dados**

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

df = pd.read_excel('/Will - Rotulados ELECTRE TRI-rC.xlsx')
df.set_index('Suppliers', inplace=True)
display(df.head())  # Mostra as primeiras linhas do DataFrame



X_criterios = df.iloc[:, 0:6].values

X_criterios

Y_classe = df.iloc[:, 6].values

Y_classe

# **Separar conjuntos de treinamento e teste**

In [ ]:
from sklearn.model_selection import train_test_split

X_criterios_treinamento, X_criterios_teste, Y_classe_treinamento, Y_classe_teste = train_test_split(X_criterios, Y_classe, test_size = 0.20, random_state = 1)


In [ ]:
Y_classe

In [ ]:
Y_classe_teste

# **KNN**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.metrics import f1_score, make_scorer, balanced_accuracy_score, matthews_corrcoef
import numpy as np

# Definindo o parâmetro de avaliação
scorer = make_scorer(f1_score, average='macro')

# Definindo a grade de parâmetros para o GridSearchCV
parametros = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'p': [1, 2]  # 1 for Manhattan distance, 2 for Euclidean distance
}

# Listas para armazenar as métricas
knn_macro_f1_scores = []
knn_micro_f1_scores = []
knn_balanced_accuracies = []
knn_mcc_scores = []
knn_best_params_list = []
knn_best_scores_list = []

for i in range(100):
    # Dividindo os dados em treinamento e teste com uma semente diferente a cada iteração
    X_criterios_treinamento, X_criterios_teste, Y_classe_treinamento, Y_classe_teste = train_test_split(
        X_criterios, Y_classe, test_size=0.20, random_state=i)

    # Configurando o StratifiedKFold
    stratified_k_fold = StratifiedKFold(n_splits=4, shuffle=True, random_state=i)

    # Executando o GridSearchCV
    grid_search = GridSearchCV(estimator=KNeighborsClassifier(),
                               param_grid=parametros,
                               scoring=scorer,
                               cv=stratified_k_fold,
                               n_jobs=-1)

    grid_search.fit(X_criterios_treinamento, Y_classe_treinamento)

    # Armazenando os melhores parâmetros e melhor resultado
    knn_melhores_parametros = grid_search.best_params_
    knn_melhor_resultado = grid_search.best_score_
    knn_best_params_list.append(knn_melhores_parametros)
    knn_best_scores_list.append(knn_melhor_resultado)

    # Melhor modelo encontrado pelo GridSearchCV
    best_knn = grid_search.best_estimator_

    # Fazendo previsões nos dados de teste
    Y_previsoes = best_knn.predict(X_criterios_teste)

    # Calculando as métricas
    knn_macro_f1 = f1_score(Y_classe_teste, Y_previsoes, average='macro')
    knn_micro_f1 = f1_score(Y_classe_teste, Y_previsoes, average='micro')
    knn_balanced_acc = balanced_accuracy_score(Y_classe_teste, Y_previsoes)
    knn_mcc = matthews_corrcoef(Y_classe_teste, Y_previsoes)

    # Armazenando as métricas
    knn_macro_f1_scores.append(knn_macro_f1)
    knn_micro_f1_scores.append(knn_micro_f1)
    knn_balanced_accuracies.append(knn_balanced_acc)
    knn_mcc_scores.append(knn_mcc)

# Calculando médias e desvios padrão das métricas
knn_mean_macro_f1 = np.mean(knn_macro_f1_scores)
knn_std_macro_f1 = np.std(knn_macro_f1_scores)
knn_mean_micro_f1 = np.mean(knn_micro_f1_scores)
knn_std_micro_f1 = np.std(knn_micro_f1_scores)
knn_mean_balanced_acc = np.mean(knn_balanced_accuracies)
knn_std_balanced_acc = np.std(knn_balanced_accuracies)
knn_mean_mcc = np.mean(knn_mcc_scores)
knn_std_mcc = np.std(knn_mcc_scores)

# Imprimindo as métricas calculadas
print(f"Média Macro F1 Score: {knn_mean_macro_f1:.4f} (Desvio Padrão: {knn_std_macro_f1:.4f})")
print(f"Média Micro F1 Score: {knn_mean_micro_f1:.4f} (Desvio Padrão: {knn_std_micro_f1:.4f})")
print(f"Média Balanced Accuracy: {knn_mean_balanced_acc:.4f} (Desvio Padrão: {knn_std_balanced_acc:.4f})")
print(f"Média MCC: {knn_mean_mcc:.4f} (Desvio Padrão: {knn_std_mcc:.4f})")

# Identificando os melhores parâmetros gerais e o melhor score
knn_best_overall_params = knn_best_params_list[np.argmax(knn_best_scores_list)]
knn_best_overall_score = max(knn_best_scores_list)

print(f"Melhores parâmetros gerais: {knn_best_overall_params}")
print(f"Melhor score geral: {knn_best_overall_score:.4f}")


In [ ]:
knn_macro_f1_scores

## **Salvando resultados**

In [ ]:
import pickle

from google.colab import drive
drive.mount('/content/drive')



In [ ]:
# Caminho no Google Drive
file_path = '/content/drive/My Drive/knn_results_DISP.pkl'

# Salvando os valores das variáveis em um arquivo
with open(file_path, 'wb') as file:
    pickle.dump({
        'knn_macro_f1_scores': knn_macro_f1_scores,
        'knn_micro_f1_scores': knn_micro_f1_scores,
        'knn_balanced_accuracies': knn_balanced_accuracies,
        'knn_mcc_scores': knn_mcc_scores,
        'knn_best_params_list': knn_best_params_list,
        'knn_best_scores_list': knn_best_scores_list,
        'knn_mean_macro_f1': knn_mean_macro_f1,
        'knn_std_macro_f1': knn_std_macro_f1,
        'knn_mean_micro_f1': knn_mean_micro_f1,
        'knn_std_micro_f1': knn_std_micro_f1,
        'knn_mean_balanced_acc': knn_mean_balanced_acc,
        'knn_std_balanced_acc': knn_std_balanced_acc,
        'knn_mean_mcc': knn_mean_mcc,
        'knn_std_mcc': knn_std_mcc,
        'knn_best_overall_params': knn_best_overall_params,
        'knn_best_overall_score': knn_best_overall_score
    }, file)


## **Carregando resultados**

In [ ]:
# Caminho no Google Drive
file_path = '/content/drive/My Drive/knn_results_DISP.pkl'

# Carregando os valores das variáveis a partir de um arquivo
try:
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
        knn_macro_f1_scores = data['knn_macro_f1_scores']
        knn_micro_f1_scores = data['knn_micro_f1_scores']
        knn_balanced_accuracies = data['knn_balanced_accuracies']
        knn_mcc_scores = data['knn_mcc_scores']
        knn_best_params_list = data['knn_best_params_list']
        knn_best_scores_list = data['knn_best_scores_list']
        knn_mean_macro_f1 = data['knn_mean_macro_f1']
        knn_std_macro_f1 = data['knn_std_macro_f1']
        knn_mean_micro_f1 = data['knn_mean_micro_f1']
        knn_std_micro_f1 = data['knn_std_micro_f1']
        knn_mean_balanced_acc = data['knn_mean_balanced_acc']
        knn_std_balanced_acc = data['knn_std_balanced_acc']
        knn_mean_mcc = data['knn_mean_mcc']
        knn_std_mcc = data['knn_std_mcc']
        knn_best_overall_params = data['knn_best_overall_params']
        knn_best_overall_score = data['knn_best_overall_score']
except FileNotFoundError:
    knn_macro_f1_scores = []
    knn_micro_f1_scores = []
    knn_balanced_accuracies = []
    knn_mcc_scores = []
    knn_best_params_list = []
    knn_best_scores_list = []
    knn_mean_macro_f1 = 0
    knn_std_macro_f1 = 0
    knn_mean_micro_f1 = 0
    knn_std_micro_f1 = 0
    knn_mean_balanced_acc = 0
    knn_std_balanced_acc = 0
    knn_mean_mcc = 0
    knn_std_mcc = 0
    knn_best_overall_params = None
    knn_best_overall_score = 0


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import f1_score, make_scorer, balanced_accuracy_score, matthews_corrcoef
import numpy as np

# Definindo o parâmetro de avaliação
scorer = make_scorer(f1_score, average='macro')

# Definindo a grade de parâmetros para o GridSearchCV
parametros = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'p': [1, 2]  # 1 for Manhattan distance, 2 for Euclidean distance
}

# Listas para armazenar as métricas
knn_macro_f1_scores = []
knn_micro_f1_scores = []
knn_balanced_accuracies = []
knn_mcc_scores = []
knn_best_params_list = []
knn_best_scores_list = []

for i in range(100):
    # Dividindo os dados em treinamento e teste com uma semente diferente a cada iteração
    X_criterios_treinamento, X_criterios_teste, Y_classe_treinamento, Y_classe_teste = train_test_split(X_criterios, Y_classe, test_size=0.20, random_state=i)

    # Executando o GridSearchCV
    grid_search = GridSearchCV(estimator=KNeighborsClassifier(),
                               param_grid=parametros,
                               scoring=scorer,
                               cv=10)

    grid_search.fit(X_criterios_treinamento, Y_classe_treinamento)

    # Armazenando os melhores parâmetros e melhor resultado
    knn_melhores_parametros = grid_search.best_params_
    knn_melhor_resultado = grid_search.best_score_
    knn_best_params_list.append(knn_melhores_parametros)
    knn_best_scores_list.append(knn_melhor_resultado)

    # Melhor modelo encontrado pelo GridSearchCV
    best_knn = grid_search.best_estimator_

    # Fazendo previsões nos dados de teste
    Y_previsoes = best_knn.predict(X_criterios_teste)

    # Calculando as métricas
    knn_macro_f1 = f1_score(Y_classe_teste, Y_previsoes, average='macro')
    knn_micro_f1 = f1_score(Y_classe_teste, Y_previsoes, average='micro')
    knn_balanced_acc = balanced_accuracy_score(Y_classe_teste, Y_previsoes)
    knn_mcc = matthews_corrcoef(Y_classe_teste, Y_previsoes)

    # Armazenando as métricas
    knn_macro_f1_scores.append(knn_macro_f1)
    knn_micro_f1_scores.append(knn_micro_f1)
    knn_balanced_accuracies.append(knn_balanced_acc)
    knn_mcc_scores.append(knn_mcc)

# Calculando médias e desvios padrão das métricas
knn_mean_macro_f1 = np.mean(knn_macro_f1_scores)
knn_std_macro_f1 = np.std(knn_macro_f1_scores)
knn_mean_micro_f1 = np.mean(knn_micro_f1_scores)
knn_std_micro_f1 = np.std(knn_micro_f1_scores)
knn_mean_balanced_acc = np.mean(knn_balanced_accuracies)
knn_std_balanced_acc = np.std(knn_balanced_accuracies)
knn_mean_mcc = np.mean(knn_mcc_scores)
knn_std_mcc = np.std(knn_mcc_scores)

# Imprimindo as métricas calculadas
print(f"Média Macro F1 Score: {knn_mean_macro_f1:.4f} (Desvio Padrão: {knn_std_macro_f1:.4f})")
print(f"Média Micro F1 Score: {knn_mean_micro_f1:.4f} (Desvio Padrão: {knn_std_micro_f1:.4f})")
print(f"Média Balanced Accuracy: {knn_mean_balanced_acc:.4f} (Desvio Padrão: {knn_std_balanced_acc:.4f})")
print(f"Média MCC: {knn_mean_mcc:.4f} (Desvio Padrão: {knn_std_mcc:.4f})")

# Identificando os melhores parâmetros gerais e o melhor score
knn_best_overall_params = knn_best_params_list[np.argmax(knn_best_scores_list)]
knn_best_overall_score = max(knn_best_scores_list)

print(f"Melhores parâmetros gerais: {knn_best_overall_params}")
print(f"Melhor score geral: {knn_best_overall_score:.4f}")

## **Salvando resultados KNN**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pickle


In [ ]:
# Salvando os valores das variáveis em um arquivo
with open('knn_results_DISP.pkl', 'wb') as file:
    pickle.dump({
        'knn_macro_f1_scores': knn_macro_f1_scores,
        'knn_micro_f1_scores': knn_micro_f1_scores,
        'knn_balanced_accuracies': knn_balanced_accuracies,
        'knn_mcc_scores': knn_mcc_scores,
        'knn_best_params_list': knn_best_params_list,
        'knn_best_scores_list': knn_best_scores_list,
        'knn_mean_macro_f1': knn_mean_macro_f1,
        'knn_std_macro_f1': knn_std_macro_f1,
        'knn_mean_micro_f1': knn_mean_micro_f1,
        'knn_std_micro_f1': knn_std_micro_f1,
        'knn_mean_balanced_acc': knn_mean_balanced_acc,
        'knn_std_balanced_acc': knn_std_balanced_acc,
        'knn_mean_mcc': knn_mean_mcc,
        'knn_std_mcc': knn_std_mcc,
        'knn_best_overall_params': knn_best_overall_params,
        'knn_best_overall_score': knn_best_overall_score
    }, file)


## **Carregando resultados KNN**

In [ ]:
# Carregando os valores das variáveis a partir de um arquivo
try:
    with open('knn_results_DISP.pkl', 'rb') as file:
        data = pickle.load(file)
        knn_macro_f1_scores = data['knn_macro_f1_scores']
        knn_micro_f1_scores = data['knn_micro_f1_scores']
        knn_balanced_accuracies = data['knn_balanced_accuracies']
        knn_mcc_scores = data['knn_mcc_scores']
        knn_best_params_list = data['knn_best_params_list']
        knn_best_scores_list = data['knn_best_scores_list']
        knn_mean_macro_f1 = data['knn_mean_macro_f1']
        knn_std_macro_f1 = data['knn_std_macro_f1']
        knn_mean_micro_f1 = data['knn_mean_micro_f1']
        knn_std_micro_f1 = data['knn_std_micro_f1']
        knn_mean_balanced_acc = data['knn_mean_balanced_acc']
        knn_std_balanced_acc = data['knn_std_balanced_acc']
        knn_mean_mcc = data['knn_mean_mcc']
        knn_std_mcc = data['knn_std_mcc']
        knn_best_overall_params = data['knn_best_overall_params']
        knn_best_overall_score = data['knn_best_overall_score']
except FileNotFoundError:
    knn_macro_f1_scores = []
    knn_micro_f1_scores = []
    knn_balanced_accuracies = []
    knn_mcc_scores = []
    knn_best_params_list = []
    knn_best_scores_list = []
    knn_mean_macro_f1 = 0
    knn_std_macro_f1 = 0
    knn_mean_micro_f1 = 0
    knn_std_micro_f1 = 0
    knn_mean_balanced_acc = 0
    knn_std_balanced_acc = 0
    knn_mean_mcc = 0
    knn_std_mcc = 0
    knn_best_overall_params = None
    knn_best_overall_score = 0


# **Naive Bayes**

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.metrics import f1_score, make_scorer, balanced_accuracy_score, matthews_corrcoef
import numpy as np

# Definindo o parâmetro de avaliação
scorer = make_scorer(f1_score, average='macro')

# Como o Naive Bayes não tem muitos hiperparâmetros para ajustar, podemos usar uma configuração padrão
parametros = {
    'var_smoothing': np.logspace(0, -9, num=100)
}

# Listas para armazenar as métricas
nb_macro_f1_scores = []
nb_micro_f1_scores = []
nb_balanced_accuracies = []
nb_mcc_scores = []
nb_best_params_list = []
nb_best_scores_list = []

for i in range(100):
    # Dividindo os dados em treinamento e teste com uma semente diferente a cada iteração
    X_criterios_treinamento, X_criterios_teste, Y_classe_treinamento, Y_classe_teste = train_test_split(
        X_criterios, Y_classe, test_size=0.20, random_state=i)

    # Configurando o StratifiedKFold
    stratified_k_fold = StratifiedKFold(n_splits=4, shuffle=True, random_state=i)

    # Executando o GridSearchCV
    grid_search = GridSearchCV(estimator=GaussianNB(),
                               param_grid=parametros,
                               scoring=scorer,
                               cv=stratified_k_fold,
                               n_jobs=-1)

    grid_search.fit(X_criterios_treinamento, Y_classe_treinamento)

    # Armazenando os melhores parâmetros e melhor resultado
    nb_melhores_parametros = grid_search.best_params_
    nb_melhor_resultado = grid_search.best_score_
    nb_best_params_list.append(nb_melhores_parametros)
    nb_best_scores_list.append(nb_melhor_resultado)

    # Melhor modelo encontrado pelo GridSearchCV
    best_nb = grid_search.best_estimator_

    # Fazendo previsões nos dados de teste
    Y_previsoes = best_nb.predict(X_criterios_teste)

    # Calculando as métricas
    nb_macro_f1 = f1_score(Y_classe_teste, Y_previsoes, average='macro')
    nb_micro_f1 = f1_score(Y_classe_teste, Y_previsoes, average='micro')
    nb_balanced_acc = balanced_accuracy_score(Y_classe_teste, Y_previsoes)
    nb_mcc = matthews_corrcoef(Y_classe_teste, Y_previsoes)

    # Armazenando as métricas
    nb_macro_f1_scores.append(nb_macro_f1)
    nb_micro_f1_scores.append(nb_micro_f1)
    nb_balanced_accuracies.append(nb_balanced_acc)
    nb_mcc_scores.append(nb_mcc)

# Calculando médias e desvios padrão das métricas
nb_mean_macro_f1 = np.mean(nb_macro_f1_scores)
nb_std_macro_f1 = np.std(nb_macro_f1_scores)
nb_mean_micro_f1 = np.mean(nb_micro_f1_scores)
nb_std_micro_f1 = np.std(nb_micro_f1_scores)
nb_mean_balanced_acc = np.mean(nb_balanced_accuracies)
nb_std_balanced_acc = np.std(nb_balanced_accuracies)
nb_mean_mcc = np.mean(nb_mcc_scores)
nb_std_mcc = np.std(nb_mcc_scores)

# Imprimindo as métricas calculadas
print(f"Média Macro F1 Score: {nb_mean_macro_f1:.4f} (Desvio Padrão: {nb_std_macro_f1:.4f})")
print(f"Média Micro F1 Score: {nb_mean_micro_f1:.4f} (Desvio Padrão: {nb_std_micro_f1:.4f})")
print(f"Média Balanced Accuracy: {nb_mean_balanced_acc:.4f} (Desvio Padrão: {nb_std_balanced_acc:.4f})")
print(f"Média MCC: {nb_mean_mcc:.4f} (Desvio Padrão: {nb_std_mcc:.4f})")

# Identificando os melhores parâmetros gerais e o melhor score
nb_best_overall_params = nb_best_params_list[np.argmax(nb_best_scores_list)]
nb_best_overall_score = max(nb_best_scores_list)

print(f"Melhores parâmetros gerais: {nb_best_overall_params}")
print(f"Melhor score geral: {nb_best_overall_score:.4f}")


In [ ]:
nb_macro_f1_scores

## **Salvando resultados NB**

In [ ]:
import pickle

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pickle

# Caminho no Google Drive
file_path = '/content/drive/My Drive/nb_results_DISP.pkl'

# Salvando os valores das variáveis em um arquivo
with open(file_path, 'wb') as file:
    pickle.dump({
        'nb_macro_f1_scores': nb_macro_f1_scores,
        'nb_micro_f1_scores': nb_micro_f1_scores,
        'nb_balanced_accuracies': nb_balanced_accuracies,
        'nb_mcc_scores': nb_mcc_scores,
        'nb_best_params_list': nb_best_params_list,
        'nb_best_scores_list': nb_best_scores_list,
        'nb_mean_macro_f1': nb_mean_macro_f1,
        'nb_std_macro_f1': nb_std_macro_f1,
        'nb_mean_micro_f1': nb_mean_micro_f1,
        'nb_std_micro_f1': nb_std_micro_f1,
        'nb_mean_balanced_acc': nb_mean_balanced_acc,
        'nb_std_balanced_acc': nb_std_balanced_acc,
        'nb_mean_mcc': nb_mean_mcc,
        'nb_std_mcc': nb_std_mcc,
        'nb_best_overall_params': nb_best_overall_params,
        'nb_best_overall_score': nb_best_overall_score
    }, file)


## **Carregando resultados NB**

In [ ]:
# Caminho no Google Drive
file_path = '/content/drive/My Drive/nb_results_DISP.pkl'

# Carregando os valores das variáveis a partir de um arquivo
try:
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
        nb_macro_f1_scores = data['nb_macro_f1_scores']
        nb_micro_f1_scores = data['nb_micro_f1_scores']
        nb_balanced_accuracies = data['nb_balanced_accuracies']
        nb_mcc_scores = data['nb_mcc_scores']
        nb_best_params_list = data['nb_best_params_list']
        nb_best_scores_list = data['nb_best_scores_list']
        nb_mean_macro_f1 = data['nb_mean_macro_f1']
        nb_std_macro_f1 = data['nb_std_macro_f1']
        nb_mean_micro_f1 = data['nb_mean_micro_f1']
        nb_std_micro_f1 = data['nb_std_micro_f1']
        nb_mean_balanced_acc = data['nb_mean_balanced_acc']
        nb_std_balanced_acc = data['nb_std_balanced_acc']
        nb_mean_mcc = data['nb_mean_mcc']
        nb_std_mcc = data['nb_std_mcc']
        nb_best_overall_params = data['nb_best_overall_params']
        nb_best_overall_score = data['nb_best_overall_score']
except FileNotFoundError:
    nb_macro_f1_scores = []
    nb_micro_f1_scores = []
    nb_balanced_accuracies = []
    nb_mcc_scores = []
    nb_best_params_list = []
    nb_best_scores_list = []
    nb_mean_macro_f1 = 0
    nb_std_macro_f1 = 0
    nb_mean_micro_f1 = 0
    nb_std_micro_f1 = 0
    nb_mean_balanced_acc = 0
    nb_std_balanced_acc = 0
    nb_mean_mcc = 0
    nb_std_mcc = 0
    nb_best_overall_params = None
    nb_best_overall_score = 0


In [ ]:
nb_macro_f1_scores

# **Decision Tree**

```
# Isto está formatado como código
```

**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.metrics import f1_score, make_scorer, balanced_accuracy_score, matthews_corrcoef
import numpy as np

# Definindo o parâmetro de avaliação
scorer = make_scorer(f1_score, average='macro')

# Definindo a grade de parâmetros para o GridSearchCV
parametros = {
    'criterion': ['gini', 'entropy'],
    'min_samples_split': [2, 3, 5],
    'min_samples_leaf': [1, 2, 5],
    'max_depth': [None, 10, 20, 30],
    'max_features': [None, 'sqrt', 'log2'],
    'splitter': ['best', 'random']
}

# Listas para armazenar as métricas
dt_macro_f1_scores = []
dt_micro_f1_scores = []
dt_balanced_accuracies = []
dt_mcc_scores = []
dt_best_params_list = []
dt_best_scores_list = []

for i in range(100):
    # Dividindo os dados em treinamento e teste com uma semente diferente a cada iteração
    X_criterios_treinamento, X_criterios_teste, Y_classe_treinamento, Y_classe_teste = train_test_split(
        X_criterios, Y_classe, test_size=0.20, random_state=i)

    # Configurando o StratifiedKFold
    stratified_k_fold = StratifiedKFold(n_splits=4, shuffle=True, random_state=i)

    # Executando o GridSearchCV
    grid_search = GridSearchCV(estimator=DecisionTreeClassifier(random_state=0),
                               param_grid=parametros,
                               scoring=scorer,
                               cv=stratified_k_fold,
                               n_jobs=-1)

    grid_search.fit(X_criterios_treinamento, Y_classe_treinamento)

    # Armazenando os melhores parâmetros e melhor resultado
    dt_melhores_parametros = grid_search.best_params_
    dt_melhor_resultado = grid_search.best_score_
    dt_best_params_list.append(dt_melhores_parametros)
    dt_best_scores_list.append(dt_melhor_resultado)

    # Melhor modelo encontrado pelo GridSearchCV
    best_dt = grid_search.best_estimator_

    # Fazendo previsões nos dados de teste
    Y_previsoes = best_dt.predict(X_criterios_teste)

    # Calculando as métricas
    dt_macro_f1 = f1_score(Y_classe_teste, Y_previsoes, average='macro')
    dt_micro_f1 = f1_score(Y_classe_teste, Y_previsoes, average='micro')
    dt_balanced_acc = balanced_accuracy_score(Y_classe_teste, Y_previsoes)
    dt_mcc = matthews_corrcoef(Y_classe_teste, Y_previsoes)

    # Armazenando as métricas
    dt_macro_f1_scores.append(dt_macro_f1)
    dt_micro_f1_scores.append(dt_micro_f1)
    dt_balanced_accuracies.append(dt_balanced_acc)
    dt_mcc_scores.append(dt_mcc)

# Calculando médias e desvios padrão das métricas
dt_mean_macro_f1 = np.mean(dt_macro_f1_scores)
dt_std_macro_f1 = np.std(dt_macro_f1_scores)
dt_mean_micro_f1 = np.mean(dt_micro_f1_scores)
dt_std_micro_f1 = np.std(dt_micro_f1_scores)
dt_mean_balanced_acc = np.mean(dt_balanced_accuracies)
dt_std_balanced_acc = np.std(dt_balanced_accuracies)
dt_mean_mcc = np.mean(dt_mcc_scores)
dt_std_mcc = np.std(dt_mcc_scores)

# Imprimindo as métricas calculadas
print(f"Média Macro F1 Score: {dt_mean_macro_f1:.4f} (Desvio Padrão: {dt_std_macro_f1:.4f})")
print(f"Média Micro F1 Score: {dt_mean_micro_f1:.4f} (Desvio Padrão: {dt_std_micro_f1:.4f})")
print(f"Média Balanced Accuracy: {dt_mean_balanced_acc:.4f} (Desvio Padrão: {dt_std_balanced_acc:.4f})")
print(f"Média MCC: {dt_mean_mcc:.4f} (Desvio Padrão: {dt_std_mcc:.4f})")

# Identificando os melhores parâmetros gerais e o melhor score
dt_best_overall_params = dt_best_params_list[np.argmax(dt_best_scores_list)]
dt_best_overall_score = max(dt_best_scores_list)

print(f"Melhores parâmetros gerais: {dt_best_overall_params}")
print(f"Melhor score geral: {dt_best_overall_score:.4f}")


## **Salvando resultados DT**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pickle

# Caminho no Google Drive
file_path = '/content/drive/My Drive/dt_results_DISP.pkl'

# Salvando os valores das variáveis em um arquivo
with open(file_path, 'wb') as file:
    pickle.dump({
        'dt_macro_f1_scores': dt_macro_f1_scores,
        'dt_micro_f1_scores': dt_micro_f1_scores,
        'dt_balanced_accuracies': dt_balanced_accuracies,
        'dt_mcc_scores': dt_mcc_scores,
        'dt_best_params_list': dt_best_params_list,
        'dt_best_scores_list': dt_best_scores_list,
        'dt_mean_macro_f1': dt_mean_macro_f1,
        'dt_std_macro_f1': dt_std_macro_f1,
        'dt_mean_micro_f1': dt_mean_micro_f1,
        'dt_std_micro_f1': dt_std_micro_f1,
        'dt_mean_balanced_acc': dt_mean_balanced_acc,
        'dt_std_balanced_acc': dt_std_balanced_acc,
        'dt_mean_mcc': dt_mean_mcc,
        'dt_std_mcc': dt_std_mcc,
        'dt_best_overall_params': dt_best_overall_params,
        'dt_best_overall_score': dt_best_overall_score
    }, file)


## **Carregando resultados DT**

In [ ]:
# Caminho no Google Drive
file_path = '/content/drive/My Drive/dt_results_DISP.pkl'

# Carregando os valores das variáveis a partir de um arquivo
try:
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
        dt_macro_f1_scores = data['dt_macro_f1_scores']
        dt_micro_f1_scores = data['dt_micro_f1_scores']
        dt_balanced_accuracies = data['dt_balanced_accuracies']
        dt_mcc_scores = data['dt_mcc_scores']
        dt_best_params_list = data['dt_best_params_list']
        dt_best_scores_list = data['dt_best_scores_list']
        dt_mean_macro_f1 = data['dt_mean_macro_f1']
        dt_std_macro_f1 = data['dt_std_macro_f1']
        dt_mean_micro_f1 = data['dt_mean_micro_f1']
        dt_std_micro_f1 = data['dt_std_micro_f1']
        dt_mean_balanced_acc = data['dt_mean_balanced_acc']
        dt_std_balanced_acc = data['dt_std_balanced_acc']
        dt_mean_mcc = data['dt_mean_mcc']
        dt_std_mcc = data['dt_std_mcc']
        dt_best_overall_params = data['dt_best_overall_params']
        dt_best_overall_score = data['dt_best_overall_score']
except FileNotFoundError:
    dt_macro_f1_scores = []
    dt_micro_f1_scores = []
    dt_balanced_accuracies = []
    dt_mcc_scores = []
    dt_best_params_list = []
    dt_best_scores_list = []
    dt_mean_macro_f1 = 0
    dt_std_macro_f1 = 0
    dt_mean_micro_f1 = 0
    dt_std_micro_f1 = 0
    dt_mean_balanced_acc = 0
    dt_std_balanced_acc = 0
    dt_mean_mcc = 0
    dt_std_mcc = 0
    dt_best_overall_params = None
    dt_best_overall_score = 0


# **Regressão logística**



In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.metrics import f1_score, make_scorer, balanced_accuracy_score, matthews_corrcoef
import numpy as np

# Definindo o parâmetro de avaliação
scorer = make_scorer(f1_score, average='macro')

# Definindo a grade de parâmetros para o GridSearchCV
parametros = {'tol': [0.0001, 0.00001, 0.000001],
              'C': [0.01, 0.1, 1, 1.5, 2, 5, 10, 100],
              'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}

# Listas para armazenar as métricas
lr_macro_f1_scores = []
lr_micro_f1_scores = []
lr_balanced_accuracies = []
lr_mcc_scores = []
lr_best_params_list = []
lr_best_scores_list = []

for i in range(1):
    # Dividindo os dados em treinamento e teste com uma semente diferente a cada iteração
    X_criterios_treinamento, X_criterios_teste, Y_classe_treinamento, Y_classe_teste = train_test_split(
        X_criterios, Y_classe, test_size=0.20, random_state=i)

    # Configurando o StratifiedKFold
    stratified_k_fold = StratifiedKFold(n_splits=4, shuffle=True, random_state=i)

    # Executando o GridSearchCV
    grid_search = GridSearchCV(estimator=LogisticRegression(random_state=0),
                               param_grid=parametros,
                               scoring=scorer,
                               cv=stratified_k_fold,
                               n_jobs=-1)

    grid_search.fit(X_criterios_treinamento, Y_classe_treinamento)

    # Armazenando os melhores parâmetros e melhor resultado
    lr_melhores_parametros = grid_search.best_params_
    lr_melhor_resultado = grid_search.best_score_
    lr_best_params_list.append(lr_melhores_parametros)
    lr_best_scores_list.append(lr_melhor_resultado)

    # Melhor modelo encontrado pelo GridSearchCV
    best_lr = grid_search.best_estimator_

    # Fazendo previsões nos dados de teste
    Y_previsoes = best_lr.predict(X_criterios_teste)

    # Calculando as métricas
    lr_macro_f1 = f1_score(Y_classe_teste, Y_previsoes, average='macro')
    lr_micro_f1 = f1_score(Y_classe_teste, Y_previsoes, average='micro')
    lr_balanced_acc = balanced_accuracy_score(Y_classe_teste, Y_previsoes)
    lr_mcc = matthews_corrcoef(Y_classe_teste, Y_previsoes)

    # Armazenando as métricas
    lr_macro_f1_scores.append(lr_macro_f1)
    lr_micro_f1_scores.append(lr_micro_f1)
    lr_balanced_accuracies.append(lr_balanced_acc)
    lr_mcc_scores.append(lr_mcc)

# Calculando médias e desvios padrão das métricas
lr_mean_macro_f1 = np.mean(lr_macro_f1_scores)
lr_std_macro_f1 = np.std(lr_macro_f1_scores)
lr_mean_micro_f1 = np.mean(lr_micro_f1_scores)
lr_std_micro_f1 = np.std(lr_micro_f1_scores)
lr_mean_balanced_acc = np.mean(lr_balanced_accuracies)
lr_std_balanced_acc = np.std(lr_balanced_accuracies)
lr_mean_mcc = np.mean(lr_mcc_scores)
lr_std_mcc = np.std(lr_mcc_scores)

# Imprimindo as métricas calculadas
print(f"Média Macro F1 Score: {lr_mean_macro_f1:.4f} (Desvio Padrão: {lr_std_macro_f1:.4f})")
print(f"Média Micro F1 Score: {lr_mean_micro_f1:.4f} (Desvio Padrão: {lr_std_micro_f1:.4f})")
print(f"Média Balanced Accuracy: {lr_mean_balanced_acc:.4f} (Desvio Padrão: {lr_std_balanced_acc:.4f})")
print(f"Média MCC: {lr_mean_mcc:.4f} (Desvio Padrão: {lr_std_mcc:.4f})")

# Identificando os melhores parâmetros gerais e o melhor score
lr_best_overall_params = lr_best_params_list[np.argmax(lr_best_scores_list)]
lr_best_overall_score = max(lr_best_scores_list)

print(f"Melhores parâmetros gerais: {lr_best_overall_params}")
print(f"Melhor score geral: {lr_best_overall_score:.4f}")


In [ ]:
lr_best_overall_score

## **Salvando resutados LR**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pickle

# Caminho no Google Drive
file_path = '/content/drive/My Drive/lr_results_DISP.pkl'

# Salvando os valores das variáveis em um arquivo
with open(file_path, 'wb') as file:
    pickle.dump({
        'lr_macro_f1_scores': lr_macro_f1_scores,
        'lr_micro_f1_scores': lr_micro_f1_scores,
        'lr_balanced_accuracies': lr_balanced_accuracies,
        'lr_mcc_scores': lr_mcc_scores,
        'lr_best_params_list': lr_best_params_list,
        'lr_best_scores_list': lr_best_scores_list,
        'lr_mean_macro_f1': lr_mean_macro_f1,
        'lr_std_macro_f1': lr_std_macro_f1,
        'lr_mean_micro_f1': lr_mean_micro_f1,
        'lr_std_micro_f1': lr_std_micro_f1,
        'lr_mean_balanced_acc': lr_mean_balanced_acc,
        'lr_std_balanced_acc': lr_std_balanced_acc,
        'lr_mean_mcc': lr_mean_mcc,
        'lr_std_mcc': lr_std_mcc,
        'lr_best_overall_params': lr_best_overall_params,
        'lr_best_overall_score': lr_best_overall_score
    }, file)


## **Carregando resultados LR**

In [ ]:
# Caminho no Google Drive
file_path = '/content/drive/My Drive/lr_results_DISP.pkl'

# Carregando os valores das variáveis a partir de um arquivo
try:
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
        lr_macro_f1_scores = data['lr_macro_f1_scores']
        lr_micro_f1_scores = data['lr_micro_f1_scores']
        lr_balanced_accuracies = data['lr_balanced_accuracies']
        lr_mcc_scores = data['lr_mcc_scores']
        lr_best_params_list = data['lr_best_params_list']
        lr_best_scores_list = data['lr_best_scores_list']
        lr_mean_macro_f1 = data['lr_mean_macro_f1']
        lr_std_macro_f1 = data['lr_std_macro_f1']
        lr_mean_micro_f1 = data['lr_mean_micro_f1']
        lr_std_micro_f1 = data['lr_std_micro_f1']
        lr_mean_balanced_acc = data['lr_mean_balanced_acc']
        lr_std_balanced_acc = data['lr_std_balanced_acc']
        lr_mean_mcc = data['lr_mean_mcc']
        lr_std_mcc = data['lr_std_mcc']
        lr_best_overall_params = data['lr_best_overall_params']
        lr_best_overall_score = data['lr_best_overall_score']
except FileNotFoundError:
    lr_macro_f1_scores = []
    lr_micro_f1_scores = []
    lr_balanced_accuracies = []
    lr_mcc_scores = []
    lr_best_params_list = []
    lr_best_scores_list = []
    lr_mean_macro_f1 = 0
    lr_std_macro_f1 = 0
    lr_mean_micro_f1 = 0
    lr_std_micro_f1 = 0
    lr_mean_balanced_acc = 0
    lr_std_balanced_acc = 0
    lr_mean_mcc = 0
    lr_std_mcc = 0
    lr_best_overall_params = None
    lr_best_overall_score = 0


# **Redes Neurais**

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.metrics import f1_score, make_scorer, balanced_accuracy_score, matthews_corrcoef
import numpy as np

# Definindo o parâmetro de avaliação
scorer = make_scorer(f1_score, average='macro')

# Definindo os parâmetros para a grid search
parametros = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam'],
    'alpha': [0.0001, 0.001],
    'learning_rate': ['constant', 'adaptive'],
    'max_iter': [5000],
    'learning_rate_init': [0.001, 0.01]
}

# Listas para armazenar as métricas
mlp_macro_f1_scores = []
mlp_micro_f1_scores = []
mlp_balanced_accuracies = []
mlp_mcc_scores = []
mlp_best_params_list = []
mlp_best_scores_list = []

for i in range(100):
    # Dividindo os dados em treinamento e teste com uma semente diferente a cada iteração
    X_criterios_treinamento, X_criterios_teste, Y_classe_treinamento, Y_classe_teste = train_test_split(
        X_criterios, Y_classe, test_size=0.20, random_state=i)

    # Configurando o StratifiedKFold
    stratified_k_fold = StratifiedKFold(n_splits=4, shuffle=True, random_state=i)

    # Executando o GridSearchCV
    grid_search = GridSearchCV(estimator=MLPClassifier(random_state=0, tol=0.0001),
                               param_grid=parametros,
                               scoring=scorer,
                               cv=stratified_k_fold,
                               n_jobs=-1)

    grid_search.fit(X_criterios_treinamento, Y_classe_treinamento)

    # Armazenando os melhores parâmetros e melhor resultado
    mlp_melhores_parametros = grid_search.best_params_
    mlp_melhor_resultado = grid_search.best_score_
    mlp_best_params_list.append(mlp_melhores_parametros)
    mlp_best_scores_list.append(mlp_melhor_resultado)

    # Melhor modelo encontrado pelo GridSearchCV
    best_mlp = grid_search.best_estimator_

    # Fazendo previsões nos dados de teste
    Y_previsoes = best_mlp.predict(X_criterios_teste)

    # Calculando as métricas
    mlp_macro_f1 = f1_score(Y_classe_teste, Y_previsoes, average='macro')
    mlp_micro_f1 = f1_score(Y_classe_teste, Y_previsoes, average='micro')
    mlp_balanced_acc = balanced_accuracy_score(Y_classe_teste, Y_previsoes)
    mlp_mcc = matthews_corrcoef(Y_classe_teste, Y_previsoes)

    # Armazenando as métricas
    mlp_macro_f1_scores.append(mlp_macro_f1)
    mlp_micro_f1_scores.append(mlp_micro_f1)
    mlp_balanced_accuracies.append(mlp_balanced_acc)
    mlp_mcc_scores.append(mlp_mcc)

# Calculando médias e desvios padrão das métricas
mlp_mean_macro_f1 = np.mean(mlp_macro_f1_scores)
mlp_std_macro_f1 = np.std(mlp_macro_f1_scores)
mlp_mean_micro_f1 = np.mean(mlp_micro_f1_scores)
mlp_std_micro_f1 = np.std(mlp_micro_f1_scores)
mlp_mean_balanced_acc = np.mean(mlp_balanced_accuracies)
mlp_std_balanced_acc = np.std(mlp_balanced_accuracies)
mlp_mean_mcc = np.mean(mlp_mcc_scores)
mlp_std_mcc = np.std(mlp_mcc_scores)

# Imprimindo as métricas calculadas
print(f"Média Macro F1 Score: {mlp_mean_macro_f1:.4f} (Desvio Padrão: {mlp_std_macro_f1:.4f})")
print(f"Média Micro F1 Score: {mlp_mean_micro_f1:.4f} (Desvio Padrão: {mlp_std_micro_f1:.4f})")
print(f"Média Balanced Accuracy: {mlp_mean_balanced_acc:.4f} (Desvio Padrão: {mlp_std_balanced_acc:.4f})")
print(f"Média MCC: {mlp_mean_mcc:.4f} (Desvio Padrão: {mlp_std_mcc:.4f})")

# Identificando os melhores parâmetros gerais e o melhor score
mlp_best_overall_params = mlp_best_params_list[np.argmax(mlp_best_scores_list)]
mlp_best_overall_score = max(mlp_best_scores_list)

print(f"Melhores parâmetros gerais: {mlp_best_overall_params}")
print(f"Melhor score geral: {mlp_best_overall_score:.4f}")


## **Salvando resultados MLP**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pickle

# Caminho no Google Drive
file_path = '/content/drive/My Drive/mlp_results_DISP.pkl'

# Salvando os valores das variáveis em um arquivo
with open(file_path, 'wb') as file:
    pickle.dump({
        'mlp_macro_f1_scores': mlp_macro_f1_scores,
        'mlp_micro_f1_scores': mlp_micro_f1_scores,
        'mlp_balanced_accuracies': mlp_balanced_accuracies,
        'mlp_mcc_scores': mlp_mcc_scores,
        'mlp_best_params_list': mlp_best_params_list,
        'mlp_best_scores_list': mlp_best_scores_list,
        'mlp_mean_macro_f1': mlp_mean_macro_f1,
        'mlp_std_macro_f1': mlp_std_macro_f1,
        'mlp_mean_micro_f1': mlp_mean_micro_f1,
        'mlp_std_micro_f1': mlp_std_micro_f1,
        'mlp_mean_balanced_acc': mlp_mean_balanced_acc,
        'mlp_std_balanced_acc': mlp_std_balanced_acc,
        'mlp_mean_mcc': mlp_mean_mcc,
        'mlp_std_mcc': mlp_std_mcc,
        'mlp_best_overall_params': mlp_best_overall_params,
        'mlp_best_overall_score': mlp_best_overall_score
    }, file)


## **Carregando resultados MLP**

In [ ]:
# Caminho no Google Drive
file_path = '/content/drive/My Drive/mlp_results_DISP.pkl'

# Carregando os valores das variáveis a partir de um arquivo
try:
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
        mlp_macro_f1_scores = data['mlp_macro_f1_scores']
        mlp_micro_f1_scores = data['mlp_micro_f1_scores']
        mlp_balanced_accuracies = data['mlp_balanced_accuracies']
        mlp_mcc_scores = data['mlp_mcc_scores']
        mlp_best_params_list = data['mlp_best_params_list']
        mlp_best_scores_list = data['mlp_best_scores_list']
        mlp_mean_macro_f1 = data['mlp_mean_macro_f1']
        mlp_std_macro_f1 = data['mlp_std_macro_f1']
        mlp_mean_micro_f1 = data['mlp_mean_micro_f1']
        mlp_std_micro_f1 = data['mlp_std_micro_f1']
        mlp_mean_balanced_acc = data['mlp_mean_balanced_acc']
        mlp_std_balanced_acc = data['mlp_std_balanced_acc']
        mlp_mean_mcc = data['mlp_mean_mcc']
        mlp_std_mcc = data['mlp_std_mcc']
        mlp_best_overall_params = data['mlp_best_overall_params']
        mlp_best_overall_score = data['mlp_best_overall_score']
except FileNotFoundError:
    mlp_macro_f1_scores = []
    mlp_micro_f1_scores = []
    mlp_balanced_accuracies = []
    mlp_mcc_scores = []
    mlp_best_params_list = []
    mlp_best_scores_list = []
    mlp_mean_macro_f1 = 0
    mlp_std_macro_f1 = 0
    mlp_mean_micro_f1 = 0
    mlp_std_micro_f1 = 0
    mlp_mean_balanced_acc = 0
    mlp_std_balanced_acc = 0
    mlp_mean_mcc = 0
    mlp_std_mcc = 0
    mlp_best_overall_params = None
    mlp_best_overall_score = 0


# **SVM**

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.metrics import f1_score, make_scorer, balanced_accuracy_score, matthews_corrcoef
import numpy as np

# Definindo o parâmetro de avaliação
macro_f1_scorer = make_scorer(f1_score, average='macro')

# Definindo a grade de parâmetros para o GridSearchCV
parametros = {
    'tol': [0.01, 0.001, 0.0005, 0.0001, 0.00005, 0.00001],
    'C': [0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 4.0, 5.0],
    'kernel': ['rbf', 'linear', 'poly', 'sigmoid'],
    'degree': [1, 2, 3, 4, 5, 6, 7, 8]
}

# Listas para armazenar as métricas
svc_macro_f1_scores = []
svc_micro_f1_scores = []
svc_balanced_accuracies = []
svc_mcc_scores = []
svc_best_params_list = []
svc_best_scores_list = []

for i in range(100):
    # Dividindo os dados em treinamento e teste com uma semente diferente a cada iteração
    X_criterios_treinamento, X_criterios_teste, Y_classe_treinamento, Y_classe_teste = train_test_split(
        X_criterios, Y_classe, test_size=0.20, random_state=i)

    # Configurando o StratifiedKFold
    stratified_k_fold = StratifiedKFold(n_splits=4, shuffle=True, random_state=i)

    # Executando o GridSearchCV
    grid_search = GridSearchCV(estimator=SVC(probability=True, random_state=0),
                               param_grid=parametros, cv=stratified_k_fold, scoring=macro_f1_scorer, n_jobs=-1)

    grid_search.fit(X_criterios_treinamento, Y_classe_treinamento)

    # Armazenando os melhores parâmetros e melhor resultado
    svc_melhores_parametros = grid_search.best_params_
    svc_melhor_resultado = grid_search.best_score_
    svc_best_params_list.append(svc_melhores_parametros)
    svc_best_scores_list.append(svc_melhor_resultado)

    # Melhor modelo encontrado pelo GridSearchCV
    best_svc = grid_search.best_estimator_

    # Fazendo previsões nos dados de teste
    Y_previsoes = best_svc.predict(X_criterios_teste)

    # Calculando as métricas
    svc_macro_f1 = f1_score(Y_classe_teste, Y_previsoes, average='macro')
    svc_micro_f1 = f1_score(Y_classe_teste, Y_previsoes, average='micro')
    svc_balanced_acc = balanced_accuracy_score(Y_classe_teste, Y_previsoes)
    svc_mcc = matthews_corrcoef(Y_classe_teste, Y_previsoes)

    # Armazenando as métricas
    svc_macro_f1_scores.append(svc_macro_f1)
    svc_micro_f1_scores.append(svc_micro_f1)
    svc_balanced_accuracies.append(svc_balanced_acc)
    svc_mcc_scores.append(svc_mcc)

# Calculando médias e desvios padrão das métricas
svc_mean_macro_f1 = np.mean(svc_macro_f1_scores)
svc_std_macro_f1 = np.std(svc_macro_f1_scores)
svc_mean_micro_f1 = np.mean(svc_micro_f1_scores)
svc_std_micro_f1 = np.std(svc_micro_f1_scores)
svc_mean_balanced_acc = np.mean(svc_balanced_accuracies)
svc_std_balanced_acc = np.std(svc_balanced_accuracies)
svc_mean_mcc = np.mean(svc_mcc_scores)
svc_std_mcc = np.std(svc_mcc_scores)

# Imprimindo as métricas calculadas
print(f"Média Macro F1 Score: {svc_mean_macro_f1:.4f} (Desvio Padrão: {svc_std_macro_f1:.4f})")
print(f"Média Micro F1 Score: {svc_mean_micro_f1:.4f} (Desvio Padrão: {svc_std_micro_f1:.4f})")
print(f"Média Balanced Accuracy: {svc_mean_balanced_acc:.4f} (Desvio Padrão: {svc_std_balanced_acc:.4f})")
print(f"Média MCC: {svc_mean_mcc:.4f} (Desvio Padrão: {svc_std_mcc:.4f})")

# Identificando os melhores parâmetros gerais e o melhor score
svc_best_overall_params = svc_best_params_list[np.argmax(svc_best_scores_list)]
svc_best_overall_score = max(svc_best_scores_list)

print(f"Melhores parâmetros gerais: {svc_best_overall_params}")
print(f"Melhor score geral: {svc_best_overall_score:.4f}")


## **Salvando resultados SVM**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import pickle

# Caminho no Google Drive
file_path = '/content/drive/My Drive/svc_results_DISP.pkl'

# Salvando os valores das variáveis em um arquivo
with open(file_path, 'wb') as file:
    pickle.dump({
        'svc_macro_f1_scores': svc_macro_f1_scores,
        'svc_micro_f1_scores': svc_micro_f1_scores,
        'svc_balanced_accuracies': svc_balanced_accuracies,
        'svc_mcc_scores': svc_mcc_scores,
        'svc_best_params_list': svc_best_params_list,
        'svc_best_scores_list': svc_best_scores_list,
        'svc_mean_macro_f1': svc_mean_macro_f1,
        'svc_std_macro_f1': svc_std_macro_f1,
        'svc_mean_micro_f1': svc_mean_micro_f1,
        'svc_std_micro_f1': svc_std_micro_f1,
        'svc_mean_balanced_acc': svc_mean_balanced_acc,
        'svc_std_balanced_acc': svc_std_balanced_acc,
        'svc_mean_mcc': svc_mean_mcc,
        'svc_std_mcc': svc_std_mcc,
        'svc_best_overall_params': svc_best_overall_params,
        'svc_best_overall_score': svc_best_overall_score
    }, file)


## **Carregando resultados SVM**

In [ ]:
# Caminho no Google Drive
file_path = '/content/drive/My Drive/svc_results_DISP.pkl'

# Carregando os valores das variáveis a partir de um arquivo
try:
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
        svc_macro_f1_scores = data['svc_macro_f1_scores']
        svc_micro_f1_scores = data['svc_micro_f1_scores']
        svc_balanced_accuracies = data['svc_balanced_accuracies']
        svc_mcc_scores = data['svc_mcc_scores']
        svc_best_params_list = data['svc_best_params_list']
        svc_best_scores_list = data['svc_best_scores_list']
        svc_mean_macro_f1 = data['svc_mean_macro_f1']
        svc_std_macro_f1 = data['svc_std_macro_f1']
        svc_mean_micro_f1 = data['svc_mean_micro_f1']
        svc_std_micro_f1 = data['svc_std_micro_f1']
        svc_mean_balanced_acc = data['svc_mean_balanced_acc']
        svc_std_balanced_acc = data['svc_std_balanced_acc']
        svc_mean_mcc = data['svc_mean_mcc']
        svc_std_mcc = data['svc_std_mcc']
        svc_best_overall_params = data['svc_best_overall_params']
        svc_best_overall_score = data['svc_best_overall_score']
except FileNotFoundError:
    svc_macro_f1_scores = []
    svc_micro_f1_scores = []
    svc_balanced_accuracies = []
    svc_mcc_scores = []
    svc_best_params_list = []
    svc_best_scores_list = []
    svc_mean_macro_f1 = 0
    svc_std_macro_f1 = 0
    svc_mean_micro_f1 = 0
    svc_std_micro_f1 = 0
    svc_mean_balanced_acc = 0
    svc_std_balanced_acc = 0
    svc_mean_mcc = 0
    svc_std_mcc = 0
    svc_best_overall_params = None
    svc_best_overall_score = 0


## **Salvando resultados RF**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import pickle

# Caminho no Google Drive
file_path = '/content/drive/My Drive/rf_results_DISP.pkl'

# Salvando os valores das variáveis em um arquivo
with open(file_path, 'wb') as file:
    pickle.dump({
        'rf_macro_f1_scores': rf_macro_f1_scores,
        'rf_micro_f1_scores': rf_micro_f1_scores,
        'rf_balanced_accuracies': rf_balanced_accuracies,
        'rf_mcc_scores': rf_mcc_scores,
        'rf_best_params_list': rf_best_params_list,
        'rf_best_scores_list': rf_best_scores_list,
        'rf_mean_macro_f1': rf_mean_macro_f1,
        'rf_std_macro_f1': rf_std_macro_f1,
        'rf_mean_micro_f1': rf_mean_micro_f1,
        'rf_std_micro_f1': rf_std_micro_f1,
        'rf_mean_balanced_acc': rf_mean_balanced_acc,
        'rf_std_balanced_acc': rf_std_balanced_acc,
        'rf_mean_mcc': rf_mean_mcc,
        'rf_std_mcc': rf_std_mcc,
        'rf_best_overall_params': rf_best_overall_params,
        'rf_best_overall_score': rf_best_overall_score
    }, file)


## **Carregando resultados RF**

In [ ]:
# Caminho no Google Drive
file_path = '/content/drive/My Drive/rf_results_DISP.pkl'

# Carregando os valores das variáveis a partir de um arquivo
try:
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
        rf_macro_f1_scores = data['rf_macro_f1_scores']
        rf_micro_f1_scores = data['rf_micro_f1_scores']
        rf_balanced_accuracies = data['rf_balanced_accuracies']
        rf_mcc_scores = data['rf_mcc_scores']
        rf_best_params_list = data['rf_best_params_list']
        rf_best_scores_list = data['rf_best_scores_list']
        rf_mean_macro_f1 = data['rf_mean_macro_f1']
        rf_std_macro_f1 = data['rf_std_macro_f1']
        rf_mean_micro_f1 = data['rf_mean_micro_f1']
        rf_std_micro_f1 = data['rf_std_micro_f1']
        rf_mean_balanced_acc = data['rf_mean_balanced_acc']
        rf_std_balanced_acc = data['rf_std_balanced_acc']
        rf_mean_mcc = data['rf_mean_mcc']
        rf_std_mcc = data['rf_std_mcc']
        rf_best_overall_params = data['rf_best_overall_params']
        rf_best_overall_score = data['rf_best_overall_score']
except FileNotFoundError:
    rf_macro_f1_scores = []
    rf_micro_f1_scores = []
    rf_balanced_accuracies = []
    rf_mcc_scores = []
    rf_best_params_list = []
    rf_best_scores_list = []
    rf_mean_macro_f1 = 0
    rf_std_macro_f1 = 0
    rf_mean_micro_f1 = 0
    rf_std_micro_f1 = 0
    rf_mean_balanced_acc = 0
    rf_std_balanced_acc = 0
    rf_mean_mcc = 0
    rf_std_mcc = 0
    rf_best_overall_params = None
    rf_best_overall_score = 0


In [ ]:
rf_mean_macro_f1